In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy.stats import multivariate_normal
from sklearn.feature_extraction.text import TfidfTransformer
%matplotlib inline

# Problem 1: Centering and Ridge Regression

In this problem we will return to predicting the median home value in a given Census area by extending linear regression. The data is in housing data.mat and it comes from http://lib.stat.cmu.edu/datasets/houses.zip. There are only 8 features for each data point; you can read about the features in `housing_data_source.txt`.

### 1.a You are given a training set ${(x_i, y_i)}_{i=1}^n$. Let $X$ be the design matrix (i.e. the matrix whose $i^{\text{th}}$ row is $x_i$, and let $y$ be the column vector whose $i^{\text{th}}$ entry is $y_i$. Let $1$ be a $n \times 1$ column vector of ones.

Define $\bar{x} = \frac{1}{n} \sum_i x_i$ and $\bar{y} = \frac{1}{n} \sum_i y_i$. Assume that the input data has been centered, so that $\bar{x} = 0$. 

Show that the optimizer of $J(w, w_0) = (y - Xw - w_0 1)^\intercal (y - Xw - w_0 1) + \lambda w^\intercal w$ is given by

$\hat{w}_0 = \bar{y}$

$\hat{w} = (X^\intercal X + \lambda 1)^{-1} X^\intercal y$

**Answer:**

### 1.b) Using the result from part a,
#### 1.b.i) Implement a ridge regression model with least squares. Include your code in the submission.

In [ ]:
house = scipy.io.loadmat('./data/housing_data.mat')
house_train_yX = np.hstack((house['Ytrain'], house['Xtrain']))
np.random.shuffle(house_train_yX)
house_train_X = np.matrix(house_train_yX[:,1:])
house_train_y = np.matrix(house_train_yX[:,0]).T

house_test_X = np.matrix(house['Xvalidate'])
house_test_y = np.matrix(house['Yvalidate'])

In [ ]:
def add_ones(X):
    m = X.shape[0]
    ones = np.ones((m, 1))
    X1 = np.hstack((ones, X))
    return X1

def least_squares(X, y, lmda):
    X1 = add_ones(X)
    m, n = X1.shape
    X1_T = X1.T
    ones = np.ones((n,1))
    w = np.linalg.inv(X1_T*X1 + lmda*ones)*X1_T*y
    return w

def rss_calc(y, y_hat):
    rss = np.sum(np.square(y-y_hat))
    return rss

def predict(X, w):
    X1 = add_ones(X)
    y_hat = X1*w
    return y_hat

def kfold_cv(X, y, k):
    X_fold = np.split(x, 10)
    # write cv function

In [ ]:
w = least_squares(house_train_X, house_train_y, lmda=0.001)

#### 1.b.ii) Use 10-fold cross validation to tune the hyperparameter $\lambda$. Using the tuned value of $\lambda$, test your trained model on the validation set. What is the residual sum of squares (RSS) on the validation set? How does this compare to the result from HW3?

In [ ]:
lambdas = [0.1, 0.01, 0.001, 0.0001]

for x in lambdas:
    # write cv function

#### 1.b.iii) Plot the regression coefficients, w. How do these compare to the result from HW3?

# Problem 2: Derivation of the Bonferroni Correction
Imagine you are playing a dice game with your friends. You roll 2 dice, and to win you must roll two sixes (anything else is a loss). You are a cheater and while your friends are not looking, you roll the dice 3 times to increase your chances.

### 2.a) What is the probability of a double 6 in 1 roll?

### 2.b) What is the probability of getting a double 6 at least once if you roll your dice 3 times?

### 2.c Now, imagine you train a classifier that gets an error rate better than your competitor's error rate. You then test the significance of the results with the bootstrap method. In 1000 samples of your test set errors, 203 samples show an error rate worse than that of your competitor. What is the p-value of the test whose null hypothesis is that your performance is not different than that of your competitor (with alternative hypothesis that you are better)?

### 2.d) You are not satisfied with the result and you try 5 other classification methods. One of them gets a bootstrap p-value of 0.05. Can you assert that your result is "significantly better" than that of your competitor with 5% chance of being wrong (in the sense that we can reject the null hypothesis with risk 0.05)?

### 2.e) Derive the Bonferroni correction, m * p-value, where m is the number of models tried, for small p-values. Hint: the p-value is like the probability of your double six dice.

### 2.f) We have a classification problem of normal vs. cancer patients using gene expression data. The feature space has d=50,000 features (genes). We use each gene as a very simple classifier: the feature value is used to predict Y (cancer = +1, normal = -1) with the AUC. The p-value in this case can be computed in a closed form (this is called the Wilcoxon-Mann-Whitney test). You find a gene with pvalue 0.0001. Is this a significant gene (in the sense that it is predictive of the Y outcome)? How come the Bonferroni correction gives a value greater than 1?

# Problem 3: Independence vs. Correlation
### 3.a) Consider the random variables $X$ and $Y \in \mathbb{R}$ with the following conditions.
* $X$ and $Y$ can take values $[-1,0,1]$.
* When $X$ is $0$, $Y$ takes values $1$ and $-1$ with equal probability $(\frac{1}{2})$. When $Y$ is $0$, $X$ takes values $1$ and $-1$ with equal probability $(\frac{1}{2})$.
* $X$ and $Y$ are $0$ with equal probability $(\frac{1}{2})$.

Are $X$ and $Y$ uncorrelated? Are $X$ and $Y$ independent? Prove your assertions. Hint: Graph these points onto the Cartesian Plane. What's each point's joint probability?

**Answer:**

Yes, $X$ and $Y$ are uncorrelated. When plotted in a Cartesian plane, the data are symmetrical across the X and Y axes. However, they are not independent because the possible values for one value are dependent on the other. For example, given the conditions above, X and Y can never assume the following pairs: $[1,1], [-1,-1], [0, 0], [1,-1], [-1,1]$.

In [ ]:
x = np.array([0,0,0,0,-1,1,-1,1])
y = np.array([-1,1,-1,1,0,0,0,0])

plt.scatter(x,y)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Possible X,Y pairs')
plt.show()

### 3.b) Consider three Bernoulli random variables $B_{1}, B_{2}, B_{3}$ which take values $\{0, 1\}$ with equal probability. Lets construct the following random variables $X, Y, Z: X = B_{1} \oplus B_{2}, Y = B_{2} \oplus B_{3}, Z = B_{1} \oplus B_{3}$, where $\oplus$ indicates the XOR operator. Are $X$, $Y$, and $Z$ pairwise independent? Mutually independent? Prove it.

**Answer:**

The pairwise joint distributions do not equal the product of their respective marginal distributions, so they are not pairewise independent.

$f_{X}(0) \neq f_{Y}(0) \neq f_{Z}(0)$,

$f_{X}(1) \neq f_{Y}(1) \neq f_{Z}(1)$

They are also not mutually independent since $f_{X, Y, Z} (x,y,z) \neq f_{X}(x) \; f_Y(y) \; f_Z(z)$

### 3.c) Why are the questions above relevant to what we are learning in class? (Hint: under what condition is a problem learnable for various models we have learned about?) Describe a dataset upon which we cannot apply the methods learned in class.

# Problem 4: Isocontours of Normal Distributions

Let $f(\mu, \Sigma)$ denote the density function of a Gaussian random variable. Plot isocontours of the following functions:

### 4.a) $f(\mu, \Sigma)$, where 
$\mu = \begin{bmatrix} 1 \\ 1 \end{bmatrix}$ and 
$\Sigma = \begin{bmatrix} 2 & 0 \\ 0 & 1 \end{bmatrix}$

**Answer:**

In [ ]:
delta = 0.025
x = np.arange(-5, 5, delta)
y = np.arange(-5, 5, delta)
X, Y = np.meshgrid(x, y)
Z = mlab.bivariate_normal(X, Y, sigmax=1, sigmay=1, mux=2, muy=1, sigmaxy=0)
plt.figure()
CS = plt.contour(X, Y, Z)
plt.show()

### 4.b) $f(\mu, \Sigma)$, where 
$\mu = \begin{bmatrix} -1 \\ 2 \end{bmatrix}$ and 
$\Sigma = \begin{bmatrix} 3 & 1 \\ 1 & 2 \end{bmatrix}$

**Answer:**

In [ ]:
delta = 0.025
x = np.arange(-5, 5, delta)
y = np.arange(-5, 5, delta)
X, Y = np.meshgrid(x, y)
Z = mlab.bivariate_normal(X, Y, sigmax=np.sqrt(3), sigmay=np.sqrt(2), mux=-1, muy=2, sigmaxy=1)
plt.figure()
CS = plt.contour(X, Y, Z)
plt.show()

### 4.c) $f(\mu_{1}, \Sigma_{1}) - f(\mu_{2}, \Sigma_{2})$, where 
$\mu_{1} = \begin{bmatrix} 0 \\ 2 \end{bmatrix}$,
$\mu_{2} = \begin{bmatrix} 2 \\ 0 \end{bmatrix}$, and
$\Sigma_{1} = \Sigma_{2} = \begin{bmatrix} 1 & 1 \\ 1 & 2 \end{bmatrix}$

**Answer:**

In [ ]:
delta = 0.025
x = np.arange(-5, 5, delta)
y = np.arange(-5, 5, delta)
X, Y = np.meshgrid(x, y)
Z1 = mlab.bivariate_normal(X, Y, sigmax=1, sigmay=np.sqrt(2), mux=0, muy=2, sigmaxy=1)
Z2 = mlab.bivariate_normal(X, Y, sigmax=1, sigmay=np.sqrt(2), mux=2, muy=0, sigmaxy=1)
Z = 10.0 * (Z1 - Z2)
plt.figure()
CS = plt.contour(X, Y, Z)
plt.show()

### 4.d) $f(\mu_{1}, \Sigma_{1}) - f(\mu_{2}, \Sigma_{2})$, where 
$\mu_{1} = \begin{bmatrix} 0 \\ 2 \end{bmatrix}$,
$\mu_{2} = \begin{bmatrix} 2 \\ 0 \end{bmatrix}$, and
$\Sigma_{1} = \begin{bmatrix} 1 & 1 \\ 1 & 2 \end{bmatrix}$ and 
$\Sigma_{2} = \begin{bmatrix} 3 & 1 \\ 1 & 2 \end{bmatrix}$

**Answer:**

In [ ]:
delta = 0.025
x = np.arange(-5, 5, delta)
y = np.arange(-5, 5, delta)
X, Y = np.meshgrid(x, y)
Z1 = mlab.bivariate_normal(X, Y, sigmax=1, sigmay=np.sqrt(2), mux=0, muy=2, sigmaxy=1)
Z2 = mlab.bivariate_normal(X, Y, sigmax=np.sqrt(3), sigmay=np.sqrt(2), mux=2, muy=0, sigmaxy=1)
Z = 10.0 * (Z1 - Z2)
plt.figure()
CS = plt.contour(X, Y, Z)
plt.show()

### 4.e) $f(\mu_{1}, \Sigma_{1}) - f(\mu_{2}, \Sigma_{2})$, where 
$\mu_{1} = \begin{bmatrix} 1 \\ 1 \end{bmatrix}$,
$\mu_{2} = \begin{bmatrix} -1 \\ -1 \end{bmatrix}$, and
$\Sigma_{1} = \begin{bmatrix} 1 & 0 \\ 0 & 2 \end{bmatrix}$ and 
$\Sigma_{2} = \begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}$

**Answer:**

In [ ]:
delta = 0.025
x = np.arange(-5, 5, delta)
y = np.arange(-5, 5, delta)
X, Y = np.meshgrid(x, y)
Z1 = mlab.bivariate_normal(X, Y, sigmax=1, sigmay=np.sqrt(2), mux=1, muy=1, sigmaxy=0)
Z2 = mlab.bivariate_normal(X, Y, sigmax=np.sqrt(2), sigmay=np.sqrt(2), mux=-1, muy=-1, sigmaxy=1)
Z = 10.0 * (Z1 - Z2)
plt.figure()
CS = plt.contour(X, Y, Z)
plt.show()

# Problem 5: Visualizing Eigenvectors of Gaussian Covariance Matrix

We have two one dimensional random variables $X_{1} \sim \mathcal{N}(3, 9)$ and $X2 \sim \frac{1}{2}X_{1} + \mathcal{N}(4, 4)$, where $\mathcal{N}(\mu, \sigma^{2})$ is a Gaussian distribution with mean $\mu$ and variance $\sigma^{2}$. In software, draw $N = 100$ random samples of $X_{1}$ and of $X_{2}$.

**Answer:**

### 5.a) Compute the mean of the sampled data.

In [ ]:
X1 = np.random.normal(3, np.sqrt(9), 100)
X2 = 0.5*X1 + np.random.normal(4, np.sqrt(4), 100)

mu_x1 = np.mean(X1)
mu_x2 = np.mean(X2)

print("Mean of X1 =", round(mu_x1,2))
print("Mean of X2 =", round(mu_x2,2))

### 5.b) Compute the covariance matrix of the sampled data.

In [ ]:
covar_matrix = np.cov(X1, X2)
print(covar_matrix)

### 5.c) Compute the eigenvectors and eigenvalues of this covariance matrix.

In [ ]:
eig_value, eig_vector = np.linalg.eig(covar_matrix)

print("Eigenvectors\n", eig_vector)
print("Eigenvalues\n",  eig_value)

### 5.d) On a two dimensional grid with a horizontal axis for $X_{1}$ ranging from $[-15, 15]$ and a vertical axis for X2 ranging from $[-15, 15]$, plot the following:

* All $N = 100$ data points
* Arrows representing both covariance eigenvectors. The eigenvector arrows should originate from the mean and have magnitude equal to their corresponding eigenvalues.

In [ ]:
fig = plt.figure(figsize=(8, 8), dpi=100)
plt.scatter(X1, X2, alpha=0.5)
ax = plt.axes()
ax.set_aspect('equal')

# 1st eigenvector 1
eig1_x = eig_vector[0,0] * eig_value[0]
eig1_y = eig_vector[1,0] * eig_value[0]
ax.arrow(mu_x1, mu_x2, eig1_x, eig1_y, width=0.01, color='red')
ax.annotate('Eigenvector 1\n{}, {}'.format(round(eig1_x + mu_x1, 2), round(eig1_y + mu_x2, 2)),
            xy=(eig1_x, eig1_y), xytext=(9, 12), color='red', size=12)

# 2nd eigenvector
eig2_x = eig_vector[0,1] * eig_value[1]
eig2_y = eig_vector[1,1] * eig_value[1]
ax.arrow(mu_x1, mu_x2, eig2_x, eig2_y, width=0.01, color='red')
ax.annotate('Eigenvector 2\n{}, {}'.format(round(eig2_x + mu_x1,2), round(eig2_y + mu_x2, 2)), 
            xy=(eig2_x, eig2_y), xytext=(-2, 8), color='red', size=12)

plt.xlim(-15, 15)
plt.ylim(-15, 15)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

### 5.e) By placing the eigenvectors of the covariance matrix into the columns of a matrix $U = [v_{1} \; v_{2}]$, where $v_{1}$ is the eigenvector corresponding to the largest eigenvalue, we can use $U'$ as a rotation matrix to rotate each of our sampled points from our original $(X_{1}, X_{2})$ coordinate system to a coordinate system aligned with the eigenvectors (without the transpose, $U$ can rotate back to the original axes). Center your data points by subtracting the mean and then rotate each point by $U'$, specifcally $x_{rotated} = U'(x - \mu)$. Plot these rotated points on a new two dimensional grid with both axes ranging from $[-15,15]$.

In [ ]:
# Center data
X = np.hstack((X1 - mu_x1, X2 - mu_x2))
X.shape = (2, 100)

# Define matrix U
U = eig_vector # eigenvectors already sorted

# Rotate data
X_rotated = U.T.dot(X)

# Plot rotated matrix
fig = plt.figure(figsize=(8, 8), dpi=100)
plt.scatter(X_rotated[0], X_rotated[1], alpha=0.5)
ax = plt.axes()
ax.set_aspect('equal')
plt.xlim(-15, 15)
plt.ylim(-15, 15)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()

# Problem 6: Covariance Matrixes and Decompositions

As described in lecture, a covariance matrix $\Sigma \in \mathbb{R}^{N, N}$ for a random variable $X \in \mathbb{R}^{N}$ with the following values, where $cov(X_{i}, X_{j}) = E[(X_{i} - \mu_{i})(X_{j} - \mu_{j})]$ is the covariance between the ith and jth elements of the random vector X:

$\Sigma = \left[\begin{array}{ccc}cov(X_{1}, X_{1}) & ... & cov(X_{1}, X_{n}) \\... &  & ... \\cov(X_{n}, X_{1}) & ... & cov(X_{n}, X_{n})\end{array}\right]$

For now, we are going to leave the formal definition of covariance matrices aside and focus instead on some transformations and properties. The motivating example we will use is the N dimensional Multivariate Gaussian Distribution defined as follows:

$f(x)=\frac{1}{\sqrt{(2\pi)^N|\Sigma|}} e\left(-\frac{1}{2}({x}-{\mu})^T{\boldsymbol \Sigma}^{-1}({x}-{\mu}) \right)$

### 6.a)

### 6.b)

### 6.c)

### 6.d)

## Problem 7: Gaussian Classifiers for Digits

In this problem we will build Gaussian classifiers for digits in MNIST. More specifically, we will model each digit class as a Gaussian distribution and make our decisions on the basis of posterior probabilities. This is a generative method for classifying images where we are modelling the class conditional probabilities as normal distributions. The steps mentioned below should be done for each training set in train.mat and you need to plot a curve of error rate vs no. of training examples upon evaluating on the test set in test.mat. Submit your trained labels for the kaggle.mat dataset on the Kaggle competition website. Please use do not use the datasets that we provided in the HW1.zip folder, and only use the datasets provided in the current HW3.zip folder. We have randomized the MNIST test and training sets.

### 7.a) Taking raw pixel values as features, fit a Gaussian distribution to each digit class using maximum likelihood estimation. This involves finding the means and covariance matrices for each digit class. Say we have i.i.d observations $X_{1}...X_{n}$, what are the maximum likelihood estimates for the mean and covariance matrix of a Gaussian distribution? Are these estimators unbiased? (No need to prove for covariance)

Tip: It is a good idea to contrast normalize images before using the raw pixel values. One way of normalization is to divide the pixel values of an image by the l2 norm of its pixel values.

Part (a) Clarification:

- Please derive the MLE for the mean.
- Please show why the MLE for the mean is unbiased.
- You do NOT have to derive the MLE for the covariance matrix - just state what it is.
- You do NOT have to show why the MLE for the covariance matrix is biased/unbiased - just state whether or not this it is unbiased.

**Answer:**

The PDF for a likelihood for a normal distribution is defined by:

$f(x_1,\ldots,x_n \mid \mu,\sigma^2) = \left( \frac{1}{2\pi\sigma^2} \right)^{n/2} \exp\left(-\frac{ \sum_{i=1}^{n}(x_i-\bar{x})^2+n(\bar{x}-\mu)^2}{2\sigma^2}\right)$

Taking the log gives us:

$\log ( \mathcal{L} (\mu,\sigma)) = (-n/2)\log(2\pi\sigma^2) - \frac{1}{2\sigma^2} \sum_{i=1}^{n} (x_i-\mu)^2$

To maximize the likelihood function over the parameter $\mu$, we take the derivative and solve with respect to $\mu$:

$\begin{align}0 & = \frac{\partial}{\partial \mu} \log ( \mathcal{L} (\mu,\sigma)) = 0 - \frac{-2n(\bar{x}-\mu)}{2\sigma^2}\end{align}$. This equation is solved by: $\hat\mu = \bar{x} = \sum^n_{i=1}x_i/n$

This is the maximum of the function. The MLE for the mean is unbiased because its expectation value is equal to the parameter $\mu$:  $E \left[ \widehat\mu \right] = \mu$ 

The MLE of the covariance matrix is: $\Sigma = {1 \over n}\sum_{i=1}^n (x_i-\overline{x})^\mathrm{T}(x_i-\overline{x})$. The MLE for the covariance is biased.

### 7.b) How would you model the prior distribution for each class? Compute prior probabilities for all classes.

**Answer:**

You can calculate the priors for each class by calculating their frequencies in the data.

In [ ]:
# Load data
digit_train = scipy.io.loadmat('./data/train.mat')
digit_test  = scipy.io.loadmat('./data/test.mat')

digit_train_y = digit_train['train_labels']
digit_train_n = digit_train_y.shape[0]
digit_train_X = digit_train['train_images'].T.reshape(digit_train_n, 28*28)

digit_train_yX = np.hstack((digit_train_y, digit_train_X))
np.random.shuffle(digit_train_yX)
digit_train_y, digit_train_X = digit_train_yX[:,0], digit_train_yX[:,1:]

digit_test_y = digit_test['test_labels']
digit_test_n = digit_test_y.shape[0]
digit_test_y = digit_test_y.reshape(digit_test_n)
digit_test_X = digit_test['test_images'].reshape(digit_test_n, 28*28)

In [ ]:
priors = np.bincount(digit_train_y)
list(zip(range(10), priors/digit_train_n))

### 7.c) Visualize the covariance matrix for a particular class. Do you see any kind of structure in the matrix? What does this symbolize?

**Answer:**

The covariance matrix has a diagonal structure.

In [ ]:
def plot_covar_matrix(X, y, digit):
    X = X[y == digit].T
    cov_mat = np.cov(X)
    plt.figure(figsize=(6, 6))
    plt.imshow(cov_mat, interpolation='none')
    plt.title('Covariance Matrix For Digit: {}'.format(digit), fontsize=12)
    plt.show()

plot_covar_matrix(digit_train_X, digit_train_y, 0)

# 7.d) We will now classify digits in the test set on the basis of posterior probabilities using two different approaches:

#### 7.d.i) Define $\Sigma_{overall}$ to be the average of the covariance matrices of all the classes. We will use this matrix as an estimate of the covariance of all the classes. This amounts to modelling class conditionals as Gaussians $(\sim \mathcal{N}(\mu_{i}, \Sigma_{overall}))$ with different means and the same covariance matrix. Using this form of class conditional probabilities, classify the images in the test set into one of the 10 classes assuming $0-1$ loss and compute the error rate and plot it over the following number of randomly chosen training data points $[100, 200, 500, 1000, 2000, 5000, 10000, 30000, 60000]$. Expect some variance in your error rate for low training data scenarios. What is the form of the decision boundary in this case? Why?

**Answer:**

The decision boundary is linear since each digit class draws from a distribution with the same covariance matrix.

See Q7, part iii for $\Sigma_{overall}$ and $\Sigma_{i}$ training sample size vs. error rate plots.

In [ ]:
def sigma_overall(X, y, alpha, labels):
    X_mu_all = []
    for i in labels:
        X_i = X[y == i]
        X_mu_all.append(np.mean(X_i, axis=0))
    X_covar_all = np.cov(X.T)
    np.fill_diagonal(X_covar_all, X_covar_all.diagonal() + alpha)
    return np.array(X_mu_all), X_covar_all


def score_overall(X, mu, cov, labels):
    pred_list = []
    for i in labels:
        multivariate_fx = multivariate_normal(mean=mu[i], cov=cov)
        logpdf = multivariate_fx.logpdf(X)
        pred_list.append(logpdf)
    y_hat = np.argmax(np.array(pred_list), axis=0)
    return y_hat


def train_gaussian_overall(train_X, train_y, test_X, test_y, sample_sizes, alpha, labels):
    error_rates = []
    for s in sample_sizes:
        mu_all, covar_all = sigma_overall(train_X[:s,:], train_y[:s], alpha, labels)
        y_hat = score_overall(test_X, mu_all, covar_all, labels)
        correct = np.sum(test_y == y_hat)
        n = test_y.shape[0]
        error_rate = 1 - (correct/n)
        error_rates.append((s, error_rate))
    return error_rates

In [ ]:
sample_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, 30000, 60000]

In [ ]:
overall_error = train_gaussian_overall(digit_train_X/255, digit_train_y, 
                                       digit_test_X/255,  digit_test_y, 
                                       sample_sizes,      alpha=0.001,
                                       labels=np.arange(10))

In [ ]:
for x in overall_error:
    print("{:<7}{:<7}".format(x[0], round(x[1],3)))

#### 7.d.ii) We can also model class conditionals as $\mathcal{N}(\mu_{i}, \Sigma_{i})$, where $\Sigma_{i}$ is the estimated covariance matrix for the $i^{th}$ class. Classify images in the test set using this form of the conditional probability (assuming $0-1$ loss) and compute the error rate and plot it over the following number of randomly chosen training data points $[100, 200, 500, 1000, 2000, 5000, 10000, 30000, 60000]$. What is the form of the decision boundary in this case?

**Answer:**

The decision boundary form is quadratic. This is because each class is from a distribution with a different covariance matrix; the shape of the gaussian is different for each class.

See Q7, part iii for $\Sigma_{overall}$ and $\Sigma_{i}$ training sample size vs. error rate plots. 

In [ ]:
def sigma_indiv(X, y, alpha, labels):
    mu_indiv = []
    covar_indiv = []
    for i in labels:
        X_i = X[y == i]
        mu_indiv.append(np.mean(X_i, axis=0))
        covar = np.cov(X_i.T)
        np.fill_diagonal(covar, covar.diagonal() + alpha)
        covar_indiv.append(covar)
    return np.array(mu_indiv), np.array(covar_indiv)


def score_indiv(X, mu_indiv, covar_indiv, labels):
    pred_list = []
    for i in labels:
        multivariate_fx = multivariate_normal(mean=mu_indiv[i], cov=covar_indiv[i])
        logpdf = multivariate_fx.logpdf(X)
        pred_list.append(logpdf)
    y_hat = np.argmax(np.array(pred_list), axis=0)
    return y_hat

    
def train_gaussian_indiv(train_X, train_y, test_X, test_y, sample_sizes, alpha, labels):
    error_rates = []
    for s in sample_sizes:
        mu_indiv, covar_indiv = sigma_indiv(train_X[:s], train_y[:s], alpha, labels)
        y_hat = score_indiv(test_X, mu_indiv, covar_indiv, labels)
        correct = np.sum(test_y == y_hat)
        n = test_y.shape[0]
        error_rate = 1 - (correct/n)
        error_rates.append((s, error_rate))
    return error_rates

In [ ]:
indiv_error = train_gaussian_indiv(digit_train_X/255, digit_train_y, 
                                   digit_test_X/255,  digit_test_y, 
                                   sample_sizes,      alpha=0.001,
                                   labels=np.arange(10))

In [ ]:
for x in indiv_error:
    print("{:<7}{:<7}".format(x[0], round(x[1],3)))

#### 7.d.iii) Compare your results in parts (i) and (ii). What do you think is the source of difference in the performance?

**Answer:**

$\Sigma_{i}$ far out performed $\Sigma_{overall}$ across all training sample sizes. Class-specific covariance matrices lead to better performance than an overall covariance matrix because it is more representative of the underlying data for each class.

In [ ]:
_, overall_err = zip(*overall_error)
_, indiv_err   = zip(*indiv_error)

plt.figure(figsize=(10,6))
plt.plot(sample_sizes, overall_err, label='$\Sigma_{overall}$')
plt.plot(sample_sizes, indiv_err,   label='$\Sigma_{i}$')
plt.ylim(0,0.6)
plt.ylabel('Error rate')
plt.xlabel('Training sample size')
plt.title('Training sample size vs. Error rate')
plt.legend(loc=1, frameon=False)
plt.show()

#### 7.d.iv) Train your best classifier using train.mat and classify the images in kaggle.mat. Submit your labels to the online Kaggle competition and record your optimum prediction rate. If you used an additional featurizer, please describe your implementation. Please only use any extra "image featurizer" code on this portion of the assignment.

Note: In your submission, you need to include learning curves (error-rate vs no. of training examples) and actual error-rate values for the above two cases and short explanations for the all the questions. Also, the covariance matrices you compute using MLE might be singular (and thus non-invertible). In order to make them non-singular and positive definite, you can add a small weight to their diagonals by setting $\Sigma_{i} = \Sigma_{i} + \alpha I$, where $\alpha$ is the weight you want to add to the diagonals. You may want to use k-fold cross validation to see what the optimum "small weight" is.

**Answer:**

I achieved a INSERT_SCORE error rate on Kaggle. For normalization, I divided the pixel values by $255$ and added an $\alpha$ value of $.001$ to my diagonals. No additional featurizer was used in my Kaggle submission. My training and test set performance vs training data size is plotted above.

#### 7.e) Now that you have developed Gaussian classification for digits, lets apply this to spam. Use the training and testing data located in spam_dataset.mat to generate a set of test labels that you will submit to the online Kaggle competition and record your optimum prediction rate. If you used an additional featurizer, please describe your implementation.

Optional: If you use the default feature set, you may obtain relatively low classification rates. The TA's suggest using a bag of words model. You may download 3rd party packages if you wish. Also, normalizing your vectors like before may help.

**Answer:**

I was able to acheive a INSERT-SCORE error rate on Kaggle.

In [ ]:
spam = scipy.io.loadmat('./data/spam_data.mat')
spam_train_X = spam['training_data']
spam_train_y = spam['training_labels'][0]
spam_test_X  = spam['test_data']

In [ ]:
# Feature engineering
tf = TfidfTransformer()
spam_tfidf_train_X = tf.fit_transform(spam_train_X, spam_train_y).toarray()
spam_tfidf_test_X  = tf.transform(spam_test_X).toarray()

In [ ]:
def sigma_spam_indiv(X, y, alpha, labels):
    X_covar = {}
    X_mu = {}
    for i in labels:
        X_i = X[y == i]
        X_mu[i] = np.mean(X_i, axis=0)
        X_cov = np.cov(X_i.T)
        np.fill_diagonal(X_cov, X_cov.diagonal() + alpha)
        X_covar[i] = X_cov
    return X_mu, X_covar


def score_spam_indiv(X, mu_dict, cov_dict, labels):
    pred = {}
    for i in labels:
        multivariate_fx = multivariate_normal(mean=mu_dict[i], cov=cov_dict[i])
        pred[i] = multivariate_fx.logpdf(X)
    return np.where(pred[0] > pred[1], 0, 1)


def train_spam_indiv(train_X, train_y, test_X, alpha, labels):
    list_error = []
    mu_matrix, cov_matrix = sigma_spam_indiv(train_X, train_y, alpha, labels)
    pred_labels = score_spam_indiv(test_X, mu_matrix, cov_matrix, labels)
    return pred_labels

In [ ]:
spam_pred = train_spam_indiv(spam_tfidf_train_X, spam_train_y, 
                             spam_tfidf_test_X, alpha=0.001,
                             labels=[0,1])

In [ ]:
# For submitting to Kaggle
# sp = np.vstack((np.arange(1,len(spam_pred)+1), spam_pred)).T
# np.savetxt(fname="./spam_tfidf_prediction.csv", X=sp, fmt='%d', delimiter=',', header='Id,Category', comments='')